# Using SuperNova-Medius on SageMaker through Model Packages

This sample notebook shows you how to deploy [SuperNova-Medius](https://huggingface.co/arcee-ai/SuperNova-Medius) using Amazon SageMaker. SuperNova-Medius is a conversational model developed by [Arcee.ai](https://www.arcee.ai). 

Arcee-SuperNova-Medius is a 14B parameter language model developed by Arcee.ai, built on the Qwen2.5-14B-Instruct architecture. This unique model is the result of a cross-architecture distillation pipeline, combining knowledge from both the Qwen2.5-72B-Instruct model and the Llama-3.1-405B-Instruct model. By leveraging the strengths of these two distinct architectures, SuperNova-Medius achieves high-quality instruction-following and complex reasoning capabilities in a mid-sized, resource-efficient form.

## Use cases

SuperNova-Medius is designed to excel in a variety of business use cases, including customer support, content creation, and technical assistance, while maintaining compatibility with smaller hardware configurations. It’s an ideal solution for organizations looking for advanced capabilities without the high resource requirements of larger models like our SuperNova-70B.

## Pre-requisites
1. This notebook works for models listed in SageMaker JumpStart and on AWS Marketplace. If you'd like to deploy from AWS Marketplace, please make sure you have previously subscribed to the model. 
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.

## Contents
1. [Select model package](#1.-Select-model-package)

2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
    1. [Define the endpoint configuration](#A.-Define-the-endpoint-configuration)
    2. [Create the endpoint](#B.-Create-the-endpoint)
    3. [Define a test payload](#C.-Define-a-test-payload)
    4. [Perform real-time inference](#D.-Perform-real-time-inference)
    5. [Visualize output](#E.-Visualize-output)
    6. [Perform streaming inference](#F.-Perform-real-time-inference)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Delete the endpoint](#B.-Delete-the-endpoint)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select the model package

In [ ]:
model_package_map = {
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Tokyo
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Seoul
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Mumbai
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Singapore
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Sydney
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Canada Central
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Frankfurt
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Stockholm
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Ireland
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # London
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Paris
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # São Paulo
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # N. Virginia
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Ohio
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # N. California
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/supernova-medius-vllm-marketpl-b001f57de24c376c90536618ad543981",  # Oregon
}

In [ ]:
%%sh
pip install -qU boto3 sagemaker

In [ ]:
import datetime
import json
import pprint

import boto3
import sagemaker
from IPython.display import Markdown, display
from sagemaker import ModelPackage, get_execution_role
from sagemaker_streaming import print_event_stream

In [ ]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise "UNSUPPORTED REGION"

model_package_arn = model_package_map[region]

In [ ]:
role = get_execution_role()
sagemaker_session = sagemaker.Session()
runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

In this example, we're deploying SuperNova-Medius on a SageMaker real-time endpoint hosted on a GPU instance. If you need general information on real-time inference with Amazon SageMaker, please refer to the SageMaker [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

The endpoint runs a DJLServing [Deep Learning Container](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-models-frameworks-djl-serving.html), powered by vLLM. vLLM enables high-performance text generation for the most popular open-source language models. 

The endpoint configuration focuses on cost efficiency. It uses a [g5.12xlarge](https://aws.amazon.com/ec2/instance-types/g5/) instance. This instance has four NVDIA A10G GPU, with 96 GB of GPU RAM. SuperNova-Medius has 14 billion 16-bit parameters, which can easily fit without the need for quantization.

#### OpenAI compatibility

The [OpenAI Messages API](https://huggingface.co/docs/text-generation-inference/messages_api) is available. This alllows you to invoke the endpoint in the same way you would invoke an OpenAI model. Likewise, the output format is identical to the OpenAI models.

### A. Define the endpoint configuration

In [ ]:
model_id = "arcee-ai/SuperNova-Medius"
endpoint_name_prefix = "SuperNova-Medius"

In [ ]:
# g5 endpoint

real_time_inference_instance_type = "ml.g5.12xlarge"

model_environment = {
    "OPTION_DTYPE": "bf16",
    "OPTION_TRUST_REMOTE_CODE": "true",
    "OPTION_ROLLING_BATCH": "vllm",
    "TENSOR_PARALLEL_DEGREE": "max",
    "OPTION_MAX_ROLLING_BATCH_SIZE": "64",
}

### B. Create the endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# create a unique endpoint name
timestamp = "{:%Y-%m-%d-%H-%M-%S}".format(datetime.datetime.now())
endpoint_name = f"{model_name}-{timestamp}"
print(f"Deploying endpoint {endpoint_name}")

# deploy the model
response = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type,
    endpoint_name=endpoint_name,
    model_data_download_timeout=600,
    container_startup_health_check_timeout=600,
)

Once the endpoint is in service, you will be able to perform real-time inference.

### C. Define a test payload

In [ ]:
model_sample_input = {
    "messages": [
        {"role": "system", "content": "You are a friendly and helpful AI assistant."},
        {
            "role": "user",
            "content": "Suggest 5 names for a new neighborhood pet food store. Names should be short, fun, easy to remember, and respectful of pets. \
        Explain why customers would like them.",
        },
    ],
    "max_tokens": 1024,
}

### D. Perform real-time inference

In [ ]:
%%time
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(model_sample_input),
)

output = json.loads(response["Body"].read().decode("utf8"))

### E. Visualize output

We can print the raw JSON output in OpenAI format.

In [ ]:
pprint.pprint(output)

We can also print the generated output with Markdown formatting.

In [ ]:
display(Markdown(output["choices"][0]["message"]["content"]))

Here are some more examples. Please feel free to tweak them and add your own!

### F. Perform streaming inference

In [ ]:
prompt = """Please write a friendly marketing pitch for a new SaaS AI platform called Arcee Cloud.
We will send this pitch by email to business and technical decision-makers, so make it sound exciting yet professional.
The contact email is sales@arcee.ai. Feel free to use emojis as appropriate.
Arcee Cloud makes it simple for enterprise users to tailor open-source small language models to their own domain knowledge,
in order to build high-quality, cost-effective and secure AI solutions."""

model_sample_input = {
    "messages": [
        {
            "role": "system",
            "content": "You are a friendly and helpful Marketing Manager working at Arcee.ai.",
        },
        {"role": "user", "content": prompt},
    ],
    "max_tokens": 1024,
    "stream": True
}

response = runtime_sm_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(model_sample_input),
    ContentType='application/json'
)

print_event_stream(response['Body'])

In [ ]:
model_sample_input = {
    "messages": [
        {
            "role": "system",
            "content": "As a friendly technical assistant engineer, answer the question in detail.",
        },
        {"role": "user", "content": "Why are transformers better models than LSTM?"},
    ],
    "max_tokens": 1024,
    "stream": True
}

response = runtime_sm_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(model_sample_input),
    ContentType='application/json'
)

print_event_stream(response['Body'])

In [ ]:
model_sample_input = {
    "messages": [
        {
            "role": "system",
            "content": "You are Darlene, a friendly and helpful salesperson \
        working at Crystal River Classic Bikes, a classic motorcycle dealership in central Florida.",
        },
        {
            "role": "user",
            "content": "Using English, write a personalized customer email to get \
        them to sign up for a test ride on the new 2025 motorcycles that are visible at the dealership. \
        Tone should be warm and personal, make sure to weave in the customer information below. \
        Wyatt, your chief mechanic and road captain, has just won the 2024 State Award for Best Mechanic. \
        \
        Customer information:\
        - name: Julien \
        - last visit: 6 months ago for bike service \
        - Owns 2 bikes, a 2002 sporty bike and a 2007 cruiser \
        ",
        },
    ],
    "max_tokens": 1024,
    "stream": True
}

response = runtime_sm_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(model_sample_input),
    ContentType='application/json'
)

print_event_stream(response['Body'])

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

## 4. Clean-up

Please don't forget to run the cells below to delete all resources and avoid unecessary charges.

### A. Delete the endpoint

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()

Thank you for trying out SuperNova-Medius on SageMaker. We have only scratched the surface of what you can do with this model.

We'd be happy to hear from you, learn more about your use case, and help you build your next AI-driven solution. Please reach out to julien@arcee.ai.